# WEB SCRAPPING ASSIGNMENT-3

In [1]:
#Importing libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException

#### Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars

In [2]:
#taking input from user
search_keyword = input("Enter a product to search: ").lower()
if search_keyword[-1] != 's':
    search_keyword = search_keyword+'s'

#initializing webdriver with chromedriver.exe
driver = webdriver.Chrome('chromedriver.exe')

#initializing driver with url https://www.amazon.in
driver.get('https://www.amazon.in')

#initializing product search field
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//input[@id="twotabsearchtextbox"]'))).send_keys(search_keyword)

#clicking search button
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//input[@id="nav-search-submit-button"]'))).click()


Enter a product to search: guitar


#### Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a `dataframe and csv`. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: 
- "Brand Name", 
- "Name of the Product", 
- "Rating", 
- "No. of Ratings", 
- "Price", 
- "Return/Exchange", 
- "Expected Delivery", 
- "Availability", 
- "Other Details" and
- “Product URL”
#### In case, if any of the details are missing for any of the product then `replace it by “-“`.

In [3]:
#getting first 3 pages url
pages_url = driver.find_elements_by_xpath('//ul[@class="a-pagination"]/li/a')
if len(pages_url)>3:
    pages_url = pages_url[:3]
pages = []
for url in pages_url:
    pages.append(url.get_attribute('href'))
    
#scraping product details from each url
brands = []
titles = []
product_urls = []
ratings = []
num_ratings = []
prices = []
return_exchanges = []
expected_deliveries = []
availabilities = []
other_details = []

#Storing current window handle
original_window = driver.current_window_handle

for url in pages:
    driver.get(url)
    
    products = driver.find_elements_by_xpath('//div[@class="sg-col-inner"]')
    
    #Scraping product details
    for product in products:
        #Scraping product name and url
        try:
            title = product.find_element_by_xpath('.//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a')
            titles.append(title.text)
            product_urls.append(title.get_attribute('href'))
        except NoSuchElementException:
            titles.append('-')
            product_urls.append('-')
            
        #Scraping ratings
        try:
            rating = product.find_element_by_xpath('.//div[@class="a-row a-size-small"]/span[1]')
            ratings.append(rating.get_attribute('aria-label'))
        except NoSuchElementException:
            ratings.append('-')
            
        #Scraping number of ratings
        try:
            num_rating = product.find_element_by_xpath('.//div[@class="a-row a-size-small"]/span[2]')
            num_ratings.append(num_rating.get_attribute('aria-label'))
        except NoSuchElementException:
            num_ratings.append('-')
            
        #Scraping prices
        try:
            price = product.find_element_by_xpath('.//span[@class="a-price-whole"]')
            prices.append(price.text)
        except NoSuchElementException:
            prices.append('-')
            
        #Scraping expected deliveries
        try:
            expected_delivery = product.find_element_by_xpath('.//span[@class="a-text-bold"]')
            expected_deliveries.append(expected_delivery.text)
        except NoSuchElementException:
            expected_deliveries.append('-')
        
        #Scraping other details
        try:
            product.find_element_by_xpath('.//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a').click()
            temp_window = driver.window_handles[-1]
            driver.switch_to.window(temp_window)
            #WebDriverWait(driver,2)
            
            #Scraping brand name
            try:
                brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')
                brands.append(brand.text)
            except NoSuchElementException:
                brands.append('-')
            
            #Scraping Return/Exchange
            try:
                return_exchange = driver.find_element_by_xpath('//div[@id="RETURNS_POLICY"]')
                return_exchanges.append(return_exchange.text)
            except NoSuchElementException:
                return_exchanges.append('-')
            
            #Scraping Availabilities
            try:
                availability = driver.find_element_by_xpath('//div[@id="availability"]')
                availabilities.append(availability.text)
            except NoSuchElementException:
                availabilities.append('-')
            
            #Scraping other product details
            try:
                other_detail = driver.find_element_by_xpath('//div[@id="feature-bullets"]')
                other_details.append(other_detail.text)
            except NoSuchElementException:
                other_details.append('-')
            
            #Closing opened link window and switch back to original window
            driver.close()
            driver.switch_to.window(original_window)
        except NoSuchElementException:
            brands.append('-')
            return_exchanges.append('-')
            availabilities.append('-')
            other_details.append('-')
            
#closing driver
driver.close()

#storing scraped products into dataframe
product_details = pd.DataFrame({
    "Brand Name": brands,
    "Name of the Product": titles,
    "Rating": ratings,
    "No. of Ratings": num_ratings,
    "Price": prices,
    "Return/Exchange": return_exchanges,
    "Expected Delivery": expected_deliveries,
    "Availability": availabilities,
    "Other Details": other_details,
    "Product URL": product_urls,
})

#Storing valid product details
df_products = product_details[product_details["Name of the Product"] != '-']

#saving products to csv file
df_products.to_csv('q2_products.csv')

#Interpreting scraped products
df_products

Brand Name  \
2    Visit the JUAREZ Store   
3    Visit the JUAREZ Store   
4    Visit the Intern Store   
5    Visit the JUAREZ Store   
6    Visit the JUAREZ Store   
..                      ...   
198           Brand: Belear   
199  Visit the JUAREZ Store   
200  Visit the JUAREZ Store   
201  Visit the JUAREZ Store   
202          Brand: KINDRED   

                                   Name of the Product              Rating  \
2    Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...  3.9 out of 5 stars   
3    Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...  3.9 out of 5 stars   
4    Intern INT-38C Acoustic Guitar Kit, With Bag, ...  3.9 out of 5 stars   
5    Juarez JRZ38C Acoustic Guitar, 38 Inch Cutaway...  4.0 out of 5 stars   
6    JUAREZ JRZ38C Right Handed Acoustic Guitar wit...  3.9 out of 5 stars   
..                                                 ...                 ...   
198  Belear K-610NAT 41 Inch Natural Dreadnought Ac...                   -   
199                 JUAREZ Arpeggio Guitar JRB40LN-3TS                   -   
200  JUAREZ Fiésta 41 Inch Acoustic Guitar Cutaway ...  2.3 out of 5 stars   
201         JUAREZ Arpeggio Guitar JRA41SP-Q-BL - blue                   -   
202  KINDRED™ Musical Guitar,4-String Musical Instr...                   -   

    No. of Ratings  Price     Return/Exchange Expected Delivery Availability  \
2            8,942  6,499  7 Days Replacement  Friday, April 30    In stock.   
3            8,942  2,499  7 Days Replacement  Friday, April 30    In stock.   
4            5,115  2,249  7 Days Replacement     Sunday, May 2                
5            3,070  2,369  7 Days Replacement  Friday, April 30    In stock.   
6            2,064  2,499  7 Days Replacement  Friday, April 30    In stock.   
..             ...    ...                 ...               ...          ...   
198              -  4,599  7 Days Replacement     Sunday, May 2    In stock.   
199              -  5,289  7 Days Replacement  Friday, April 30    In stock.   
200              7  4,009  7 Days Replacement  Friday, April 30    In stock.   
201              -  6,729  7 Days Replacement  Friday, April 30    In stock.   
202              -    795  7 Days Replacement     Sunday, May 2    In stock.   

                                         Other Details  \
2    About this item\nBlack Glossy Finish, Number o...   
3    About this item\nBlack Glossy Finish, Number o...   
4    About this item\nGreat looks with an innovativ...   
5    About this item\nBlack glossy finish, number o...   
6    About this item\nNumber of Frets: 18\nAcoustic...   
..                                                 ...   
198  41 Inch Dreadnought Acoustic Guitar offers a l...   
199                       About this item\nJRB40LN-3TS   
200  About this item\nGlossy finish, Number of fret...   
201  About this item\nJRA41SP-Q-BL\nColor: Blue\nBo...   
202  The Guitar is made of very good quality plasti...   

                                           Product URL  
2    https://www.amazon.in/Juarez-Acoustic-Cutaway-...  
3    https://www.amazon.in/Juarez-Acoustic-Cutaway-...  
4    https://www.amazon.in/Intern-INT-38C-Acoustic-...  
5    https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...  
6    https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-N...  
..                                                 ...  
198  https://www.amazon.in/Belear-K-610NAT-Natural-...  
199  https://www.amazon.in/gp/slredirect/picassoRed...  
200  https://www.amazon.in/gp/slredirect/picassoRed...  
201  https://www.amazon.in/gp/slredirect/picassoRed...  
202  https://www.amazon.in/gp/slredirect/picassoRed...  

[171 rows x 10 columns]

#### Q3. Write a python program to access the search bar and search button on https://images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [5]:
#initializing webdriver with chromedriver.exe
driver = webdriver.Chrome('chromedriver.exe')

#initializing search keywords into list
search_keywords = ['fruits','cars','Machine Learning']

images = {}
for keyword in search_keywords:
    #initializing driver with url https://images.google.com
    driver.get('https://images.google.com')
    
    #initializing search field box with keyword
    WebDriverWait(driver,5).until(ec.element_to_be_clickable((By.XPATH,'//input[@class="gLFyf gsfi"]'))).send_keys(keyword)
    
    #clicking on search button
    WebDriverWait(driver,5).until(ec.element_to_be_clickable((By.XPATH,'//button[@class="Tg7LZd"]'))).click()
    
    #scrolling down to get image containers loaded on page
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    
    #initializing 100 image containers
    image_containers = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')[:100]
    
    images[keyword] = []
    for image in image_containers:
        try:
            image.click()
            images[keyword].append(image.get_attribute('src'))
        except:
            images[keyword].append('-')

#Closing driver
driver.close()

#storing images into dataframe
df_images = pd.DataFrame(images)

#interpreting scraped images
df_images

fruits  \
0   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
1   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
2   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
3   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
4   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
..                                                ...   
95  https://encrypted-tbn0.gstatic.com/images?q=tb...   
96  https://encrypted-tbn0.gstatic.com/images?q=tb...   
97  https://encrypted-tbn0.gstatic.com/images?q=tb...   
98  https://encrypted-tbn0.gstatic.com/images?q=tb...   
99  https://encrypted-tbn0.gstatic.com/images?q=tb...   

                                                 cars  \
0   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
1   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
2   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
3   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
4   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...   
..                                                ...   
95  https://encrypted-tbn0.gstatic.com/images?q=tb...   
96  https://encrypted-tbn0.gstatic.com/images?q=tb...   
97  https://encrypted-tbn0.gstatic.com/images?q=tb...   
98  https://encrypted-tbn0.gstatic.com/images?q=tb...   
99  https://encrypted-tbn0.gstatic.com/images?q=tb...   

                                     Machine Learning  
0   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...  
1   data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...  
2   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...  
3   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...  
4   data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...  
..                                                ...  
95  https://encrypted-tbn0.gstatic.com/images?q=tb...  
96  https://encrypted-tbn0.gstatic.com/images?q=tb...  
97  https://encrypted-tbn0.gstatic.com/images?q=tb...  
98                                                  -  
99  https://encrypted-tbn0.gstatic.com/images?q=tb...  

[100 rows x 3 columns]

#### Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: 
- “Brand Name”, 
- “Smartphone name”, 
- “Colour”, 
- “RAM”, 
- “Storage(ROM)”, 
- “Primary Camera”, 
- “Secondary Camera”, 
- “Display Size”, 
- “Display Resolution”, 
- “Processor”, 
- “Processor Cores”, 
- “Battery Capacity”, 
- “Price”, 
- “Product URL”. 
#### Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [6]:
#Taking search input from user
search_keyword = input("Search for Smartphone?")

#initializing webdriver with chromedriver
driver = webdriver.Chrome('chromedriver.exe')

#initialzing driver with url https://www.flipkart.com
driver.get('https://www.flipkart.com')

#closing login popup
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]'))).click()

#intializing search input box field
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//input[@class="_3704LK"]'))).send_keys(search_keyword)

#clicking on search button
driver.find_element_by_xpath('//button[@class="L0Z3Pu"]').click()

#initializing all products of first search page
try:
    WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//div[@class="_2kHMtA"]')))
    product_containers = driver.find_elements_by_xpath('//div[@class="_2kHMtA"]')
except TimeoutException:
    product_containers = driver.find_elements_by_xpath('//div[@class="_4ddWXP"]')

#storing current window handle
original_window = driver.current_window_handle

#Scraping details for each product
brand_names = []
product_names = []
colors = []
rams = []
storages = []
primary_cameras = []
secondary_cameras = []
display_sizes = []
display_resolutions = []
processors = []
processor_cores = []
battery_capacities = []
prices = []
product_urls = []
for product in product_containers:
    #scraping product urls
    try:
        p_url = product.find_element_by_xpath('.//a[1]')
        product_urls.append(p_url.get_attribute('href'))
    except NoSuchElementException:
        product_urls.append('-')
    
    #visiting product page to scrape details
    product.click()
    temp_window = driver.window_handles[-1]
    driver.switch_to.window(temp_window)
    
    #scraping product names
    try:
        p_name = driver.find_element_by_xpath('//span[@class="B_NuCI"]')
        product_names.append(p_name.text)
    except NoSuchElementException:
        product_names.append('-')
      
    #scraping product prices
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        prices.append(price.text)
    except NoSuchElementException:
        prices.append('-')
        
        
    #clicking read more button
    try:
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    except NoSuchElementException:
        pass
    
    #scraping colour
    try:
        color = driver.find_element_by_xpath('//td[normalize-space(text())="Color"]/following-sibling::td')
        colors.append(color.text)
    except NoSuchElementException:
        colors.append('-')
        
    #scraping rams
    try:
        ram = driver.find_element_by_xpath('//td[normalize-space(text())="RAM"]/following-sibling::td')
        rams.append(ram.text)
    except NoSuchElementException:
        rams.append('-')
    
    #scraping storages
    try:
        storage = driver.find_element_by_xpath('//td[normalize-space(text())="Internal Storage"]/following-sibling::td')
        storages.append(storage.text)
    except NoSuchElementException:
        storages.append('-')
        
    #scraping primary cameras
    try:
        primary_camera = driver.find_element_by_xpath('//td[normalize-space(text())="Primary Camera"]/following-sibling::td')
        primary_cameras.append(primary_camera.text)
    except NoSuchElementException:
        primary_cameras.append('-')
        
    #scraping secondary cameras
    try:
        secondary_camera = driver.find_element_by_xpath('//td[normalize-space(text())="Secondary Camera"]/following-sibling::td')
        secondary_cameras.append(secondary_camera.text)
    except NoSuchElementException:
        secondary_cameras.append('-')
        
    #scraping display sizes
    try:
        display_size = driver.find_element_by_xpath('//td[normalize-space(text())="Display Size"]/following-sibling::td')
        display_sizes.append(display_size.text)
    except NoSuchElementException:
        display_sizes.append('-')
        
    #scraping display resolutions
    try:
        display_resolution = driver.find_element_by_xpath('//td[normalize-space(text())="Resolution"]/following-sibling::td')
        display_resolutions.append(display_resolution.text)
    except:
        display_resolutions.append('-')
        
    #scraping processors
    try:
        processor = driver.find_element_by_xpath('//td[normalize-space(text())="Processor Type"]/following-sibling::td')
        processors.append(processor.text)
    except NoSuchElementException:
        processors.append('-')
        
    #scraping processor cores
    try:
        processor_core = driver.find_element_by_xpath('//td[normalize-space(text())="Processor Core"]/following-sibling::td')
        processor_cores.append(processor_core.text)
    except NoSuchElementException:
        processor_cores.append('-')
        
    #scraping battery capacities
    try:
        battery_capacity = driver.find_element_by_xpath('//td[normalize-space(text())="Battery Capacity"]/following-sibling::td')
        battery_capacities.append(battery_capacity.text)
    except NoSuchElementException:
        battery_capacities.append('-')
    
    #scraping brand names
    try:
        brand_name = driver.find_element_by_xpath('//td[normalize-space(text())="Model Name"]/following-sibling::td')
        brand_names.append(brand_name.text)
    except NoSuchElementException:
        brand_names.append('-')
        
    #closing temporary opened link window and switch back to original window
    driver.close()
    driver.switch_to.window(original_window)
    
#closing driver
driver.close()

#storing details into dataframe
df_smartphones = pd.DataFrame({
    "Brand Name": brand_names,
    "Smartphone Name": product_names,
    "Colour": colors,
    "RAM": rams,
    "Storage(ROM)": storages,
    "Primary Camera": primary_cameras,
    "Secondary Camera": secondary_cameras,
    "Display Size": display_sizes,
    "Display Resolution": display_resolutions,
    "Processor": processors,
    "Processor Core": processor_cores,
    "Battery Capacity": battery_capacities,
    "Price": prices,
    "Product URL": product_urls,
})

#saving product details to csv file
df_smartphones.to_csv('q4_smartphones.csv')

#interpreting smartphone details
df_smartphones

Search for Smartphone?pixel


Brand Name                                    Smartphone Name  \
0        Pixel 4a   Google Pixel 4a (Just Black, 128 GB)  (6 GB RAM)   
1        Pixel 3a  Google Pixel 3a (Clearly White, 64 GB)  (4 GB ...   
2   Redmi 9 Prime     Redmi 9 Prime (Matte Black, 64 GB)  (4 GB RAM)   
3         9 Power    REDMI 9 Power (Blazing Blue, 64 GB)  (4 GB RAM)   
4         9 Power  REDMI 9 Power (Electric Green, 64 GB)  (4 GB RAM)   
5         9 Power    REDMI 9 Power (Mighty Black, 64 GB)  (4 GB RAM)   
6          Note 9     REDMI Note 9 (Arctic White, 64 GB)  (4 GB RAM)   
7          Note 9       REDMI Note 9 (Aqua Green, 64 GB)  (4 GB RAM)   
8         9 Power   REDMI 9 Power (Mighty Black, 128 GB)  (6 GB RAM)   
9         9 Power       REDMI 9 Power (Fiery Red, 64 GB)  (4 GB RAM)   
10         Note 9     REDMI Note 9 (Shadow Black, 64 GB)  (4 GB RAM)   
11        Redmi 9         Redmi 9 (Sporty Orange, 64 GB)  (4 GB RAM)   
12        Redmi 9              Redmi 9 (Sky Blue, 64 GB)  (4 GB RAM)   
13        Redmi 9          Redmi 9 (Carbon Black, 64 GB)  (4 GB RAM)   
14           X744                             Micromax X744  (Green)   
15            KX3                           KARBONN KX3  (Black,Red)   
16           Y20G        ViVO Y20G (Purist Blue, 128 GB)  (6 GB RAM)   
17       Redmi 9A              Redmi 9A (SeaBlue, 32 GB)  (2 GB RAM)   
18       Redmi 9A         Redmi 9A (Nature Green, 32 GB)  (2 GB RAM)   
19       Redmi 9A       Redmi 9A (Midnight Black, 32 GB)  (2 GB RAM)   
20       M01 core        SAMSUNG M01 core (Black, 16 GB)  (1 GB RAM)   
21       M01 core         SAMSUNG M01 core (Blue, 32 GB)  (2 GB RAM)   
22        9 Power   REDMI 9 Power (Blazing Blue, 128 GB)  (4 GB RAM)   
23        VISION1  Itel VISION1 (GRADATION GREEN, 32 GB)  (2 GB RAM)   

             Colour    RAM Storage(ROM)      Primary Camera  Secondary Camera  \
0        Just Black   6 GB       128 GB  12.2MP Rear Camera  8MP Front Camera   
1     Clearly White   4 GB        64 GB  12.2MP Rear Camera  8MP Front Camera   
2       Matte Black   4 GB        64 GB    13MP Rear Camera  8MP Front Camera   
3      Blazing Blue   4 GB        64 GB    48MP Rear Camera  8MP Front Camera   
4    Electric Green   4 GB        64 GB    48MP Rear Camera  8MP Front Camera   
5      Mighty Black   4 GB        64 GB    48MP Rear Camera  8MP Front Camera   
6      Arctic White   4 GB        64 GB    48MP Rear Camera                 -   
7        Aqua Green   4 GB        64 GB    48MP Rear Camera                 -   
8      Mighty Black   6 GB       128 GB    48MP Rear Camera  8MP Front Camera   
9         Fiery Red   4 GB        64 GB    48MP Rear Camera  8MP Front Camera   
10     Shadow Black   4 GB        64 GB    48MP Rear Camera                 -   
11    Sporty Orange   4 GB        64 GB          13MP + 8MP  5MP Front Camera   
12         Sky Blue   4 GB        64 GB          13MP + 2MP  5MP Front Camera   
13     Carbon Black   4 GB        64 GB          13MP + 2MP  5MP Front Camera   
14            Green  32 MB        32 MB   0.3MP Rear Camera                 -   
15        Black,Red  32 MB        32 MB   0.3MP Rear Camera                 -   
16      Purist Blue   6 GB       128 GB    13MP Rear Camera                 -   
17          SeaBlue   2 GB        32 GB    13MP Rear Camera                 -   
18     Nature Green   2 GB        32 GB    13MP Rear Camera                 -   
19   Midnight Black   2 GB        32 GB    13MP Rear Camera                 -   
20            Black   1 GB        16 GB     8MP Rear Camera  5MP Front Camera   
21             Blue   2 GB        32 GB     8MP Rear Camera  5MP Front Camera   
22     Blazing Blue   4 GB       128 GB    48MP Rear Camera  8MP Front Camera   
23  GRADATION GREEN   2 GB        32 GB     8MP Rear Camera  5MP Front Camera   

             Display Size  Display Resolution                       Processor  \
0    14.76 cm (5.81 inch)  2340 x 1080 Pixels        Qualcomm Snapdragon 730G   
1     14.22 cm (5.

#### Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [9]:
#taking input as city name
search_city = input("Search for city: ")

#initializing webdriver with chromedriver
driver = webdriver.Chrome('chromedriver.exe')

#initializing driver with url https://map.google.com
driver.get('https://map.google.com')

#initializing search input box
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//input[@id="searchboxinput"]'))).send_keys(search_city)

#clicking search button
driver.find_element_by_xpath('//button[@id="searchbox-searchbutton"]').click()

#initializing url from page having geospatial coordinates
WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,'//div[normalize-space(text())="Share"]'))).click()
WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,'//button[normalize-space(text())="Embed a map"]'))).click()
WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,'//input[@class="section-embed-map-input"]')))
geo_url = driver.find_element_by_xpath('//input[@class="section-embed-map-input"]').get_attribute('value')

#extracting cordinates(latitude,longitude)
geo_cordinates = geo_url.split('!2d')[1].split('!2m')[0].split('!3d')
latitude = geo_cordinates[1]
logitude = geo_cordinates[0]

#closing driver
driver.close()

#Interpreting cordinates
print(f'Cordinates (latitude,logitude): ({latitude},{logitude})')

Search for city: Begusarai
Cordinates (latitude,logitude): (25.417864001078193,86.09001373329437)


#### Q6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from https://trak.in.

In [29]:
#initializing webdriver with chromedriver
driver = webdriver.Chrome('chromedriver.exe')

#initializing driver with url https://trak.in
driver.get('https://trak.in')

#clicking on link Funding Deals
try:
    WebDriverWait(driver,5).until(ec.element_to_be_clickable((By.XPATH,'//a[contains(text(),"Funding Deals")]'))).click()
except TimeoutException:
    WebDriverWait(driver,5).until(ec.element_to_be_clickable((By.XPATH,'//a[text()="More"]'))).click()
    WebDriverWait(driver,5).until(ec.element_to_be_clickable((By.XPATH,'//a[contains(text(),"Funding Deals")]'))).click()
    #after this ad pop up needs to be closed manually
    
    
#initializing second quarter as a list and scraping the funding deals containers into list
funding_deals_containers = []
second_quarter = ['July, 2020','August, 2020','September, 2020']
driver.switch_to.default_content()
for month in second_quarter:
    container = WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,f'//h2[normalize-space(text())="{month}"]/following-sibling::div')))
    funding_deals_containers.append(container)

funding_deals_containers

[<selenium.webdriver.remote.webelement.WebElement (session="41c6d70ea108d713142683de4b46dfe1", element="8abca115-b95b-45be-9268-fb45834f7774")>,
 <selenium.webdriver.remote.webelement.WebElement (session="41c6d70ea108d713142683de4b46dfe1", element="4e05b807-0ea1-4410-8f4b-7421032a31e2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="41c6d70ea108d713142683de4b46dfe1", element="7b570083-c978-4d77-95b7-119a67447e8f")>]

In [33]:
#Scraping data
serial_numbers = []
dates = []
startup_names = []
industries = []
sub_verticals = []
locations = []
investor_names = []
investment_types = []
amounts = []

for fund in funding_deals_containers:
    rows = fund.find_elements_by_xpath('.//div[@class="dataTables_scrollBody"]/table/tbody/tr')
    for row in rows:
        cols = row.find_elements_by_xpath('.//td')
        #storing data
        serial_numbers.append(cols[0].text)
        dates.append(cols[1].text)
        startup_names.append(cols[2].text)
        industries.append(cols[3].text)
        sub_verticals.append(cols[4].text)
        locations.append(cols[5].text)
        investor_names.append(cols[6].text)
        investment_types.append(cols[7].text)
        amounts.append(cols[8].text)


#closing driver
driver.close()

#storing records in dataframe        
df_funding_deals = pd.DataFrame({
    "S.No.": serial_numbers,
    "Date": dates,
    "Startup Name": startup_names,
    "Industry/Vertical": industries,
    "Sub-Vertical": sub_verticals,
    "City/Location": locations,
    "Inverstor's Name": investor_names,
    "Investment Type": investment_types,
    "Amount (in USD)": amounts,
})

#Interpreting Scraped Funding Deals
df_funding_deals

S.No.        Date               Startup Name  \
0      1  15/07/2020                   Flipkart   
1      2  16/07/2020                    Vedantu   
2      3  16/07/2020                       Crio   
3      4  14/07/2020                    goDutch   
4      5  13/07/2020                   Mystifly   
5      6  09/07/2020               JetSynthesys   
6      7  10/07/2020                   gigIndia   
7      8  15/07/2020                  PumPumPum   
8      9  14/07/2020                       FLYX   
9     10  13/07/2020  Open Appliances Pvt. Ltd.   
10     1  15/08/2020                     Practo   
11     2  13/08/2020                    Medlife   
12     3  13/08/2020                  HungerBox   
13     4  04/08/2020                      Dunzo   
14     5  11/08/2020                   Terra.do   
15     6  12/08/2020                  Classplus   
16     7  14/08/2020                       Niyo   
17     8  10/08/2020                  ZestMoney   
18     9  07/08/2020                FreshToHome   
19    10  13/08/2020                    Eduvanz   
20     1  08/09/2020                     Byju’s   
21     2  12/09/2020                  mCaffeine   
22     3  09/09/2020                     Qshala   
23     4  02/09/2020                      Winzo   
24     5  09/09/2020                Hippo Video   
25     6  07/09/2020                    Melorra   
26     7  07/09/2020                        1mg   
27     8  31/08/2020                      mfine   
28     9  31/08/2020                       Apna   
29    10  03/09/2020                    Railofy   

                         Industry/Vertical  \
0                               E-commerce   
1                                  EduTech   
2                                  EduTech   
3                                  FinTech   
4                      Airfare Marketplace   
5                 Gaming and Entertainment   
6                              Marketplace   
7                        Automotive Rental   
8                               OTT Player   
9                   Information Technology   
10                              HealthTech   
11                              E-commerce   
12                                FoodTech   
13                   Hyper-local Logistics   
14                                 EduTech   
15                                 EduTech   
16                                 FinTech   
17                                 FinTech   
18                              E-commerce   
19                                 FinTech   
20                                 EduTech   
21                           Personal Care   
22                                 EduTech   
23                           Online Gaming   
24  Video Customer Experience(CX) Platform   
25                              E-commerce   
26                              E-commerce   
27                              HealthTech   
28                         Human Resources   
29                          Transportation   

                                         Sub-Vertical  \
0                                          E-commerce   
1                                     Online Tutoring   
2                    Learning Platform for Developers   
3                                      Group Payments   
4   Ticketing, Airline Retailing, and Post-Ticketi...   
5                            Gaming and Entertainment   
6                           Crowd Sourcing, Freelance   
7                           Used Car-leasing platform   
8                            Streaming Social Network   
9               Internet-of-Things Security Solutions   
10                           Health care and Wellness   
11                                    Online Pharmacy   
12                       Online Food Delivery Service   
13                           Online Delivery Services   
14                  Online Climate School, E-learning   
15                        E-learning, Online Tutoring   
16                       

#### Q7. Write a program to scrap all the available details of top 10 gaming laptops from https://digit.in.

In [37]:
from selenium.webdriver.common.keys import Keys
#initializing webdriver with chromedriver
driver = webdriver.Chrome('chromedriver.exe')

#initializing driver with url https://digit.in
driver.get('https://digit.in')

#searching for top 10 gaming laptops
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//div[@class="search"]/a'))).click()
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//input[@id="globalPageSearchText"]'))).send_keys('top 10 gaming laptops')
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//input[@id="globalPageSearchText"]'))).send_keys(Keys.ENTER)


In [39]:
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//input[@id="content_top10"]'))).click()
WebDriverWait(driver,10).until(ec.element_to_be_clickable((By.XPATH,'//div[normalize-space(text())="Best Gaming Laptops in India"]'))).click()

In [43]:
#Scraping top 10 laptop names
title_containers = driver.find_elements_by_class_name('TopNumbeHeading')
titles = []
for title in title_containers:
    titles.append(title.find_element_by_xpath('.//a').text)

In [45]:
#Scraping top 10 laptop feature details
operating_systems = []
displays = []
processors = []
memories = []
feature_containers = driver.find_elements_by_xpath('//div[@class="Specs-Wrap"]/ul')
for feature in feature_containers:
    all_specs = feature.find_elements_by_xpath('.//div[@class="value"]')
    operating_systems.append(all_specs[0].text)
    displays.append(all_specs[1].text)
    processors.append(all_specs[2].text)
    memories.append(all_specs[3].text)

In [46]:
#Scraping seller names
seller_names = []
seller_containers = driver.find_elements_by_xpath('//td[@class="smmerchant"]')
for seller in seller_containers:
    seller_names.append(seller.text)

In [47]:
#scraping laptop prices
prices = []
price_containers = driver.find_elements_by_xpath('//td[@class="smprice"]')
for price in price_containers:
    prices.append(price.text)

In [48]:
#storing scraped details in dataframe
df_top_10_glaptops = pd.DataFrame({
    "Laptop Name": titles,
    "OS": operating_systems,
    "Display": displays,
    "Processor": processors,
    "Memory": memories,
    "Seller": seller_names,
    "Price": prices
})

#Interpreting scraped top 10 gamming laptops
df_top_10_glaptops

Laptop Name                      OS  \
0      MSI GT76 TITAN DT 9SG          WINDOWS 10 PRO   
1      ALIENWARE 17 AREA-51M          WINDOWS 10 PRO   
2            HP OMEN 15 2020         WINDOWS 10 HOME   
3          ASUS ZEPHYRUS G14         WINDOWS 10 HOME   
4         LENOVO LEGION Y540         WINDOWS 10 HOME   
5  ASUS ROG ZEPHYRUS G GA502         WINDOWS 10 HOME   
6  ASUS ROG ZEPHYRUS S GX531         WINDOWS 10 HOME   
7   MSI GT83VR 7RE TITAN SLI  WINDOWS 10 HOME 64 BIT   
8   ASUS ROG ZEPHYRUS DUO 15              WINDOWS 10   
9                    DELL G3    WINDOWS 10 HOME PLUS   

                     Display  \
0      17.3" (UHD 3840X2160)   
1  17.3" (FHD (1920 X 1080))   
2        15.6" (1920 X 1080)   
3          14" (1920 X 1080)   
4        15.6" (1920 X 1080)   
5        15.6" (1920 X 1080)   
6        15.6" (1920 X 1080)   
7        18.4" (1920 X 1080)   
8        15.6" (3840 X 1100)   
9               15.6 MP | NA   

                                           Processor                   Memory  \
0             INTEL 9TH GEN CORE I9-9900K | 5000 MHZ    1 TB HDD/64 GBGB DDR4   
1             INTEL 9TH GEN CORE I9-9900K | 5000 MHZ  1 TB PCIE SSD/32GB DDR4   
2                  INTEL I7-10750H 10TH GEN | 1.6GHZ  512 GB SSD/16 GBGB DDR4   
3               AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ    1 TB SSD/16 GBGB DDR4   
4       9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ        1 TB SSD/8GB DDR4   
5              AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ     512 GB SSD/16GB DDR4   
6              9TH GEN INTEL CORE I7-8750H | 2.2 GHZ   512GB SSD/16 GBGB DDR4   
7  INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ    1.5 TB SATA/64GB DDR4   
8                 INTEL CORE I7 10TH GEN 10875H | NA   512 GB SSD/4 GBGB DDR4   
9               8TH GEN INTEL CORE I5-8300H | 2.3GHZ         1TB HDD/8GB DDR4   

     Seller    Price  
0       N/A  ₹379990  
1    amazon  ₹422000  
2  Tatacliq  ₹117790  
3    amazon  ₹164990  
4  flipkart   ₹79990  
5  flipkart   ₹79990  
6  flipkart  ₹239990  
7       N/A  ₹349990  
8    amazon  ₹244990  
9    amazon   ₹61990